In [1]:
library(Seurat)
library(trendsceek)
library(data.table)

Loading required package: ggplot2
Loading required package: cowplot

Attaching package: ‘cowplot’

The following object is masked from ‘package:ggplot2’:

    ggsave

Loading required package: Matrix
Warning message:
“package ‘data.table’ was built under R version 3.4.4”

In [ ]:
for (i in 1:4){
Rep <-read.table(paste0('../Data/Layer',i,'1_BC_count_matrix-1.tsv'),sep="\t",header = T,row.names = 1)
Rep <-t(Rep)

Rep<-genefilter_exprmat(Rep ,min.expr=5,min.ncells.expr = 3)
Rep <-CreateSeuratObject(raw.data = Rep ,min.cells = 3,min.genes = 5 )
Rep_filt<- FilterCells(Rep, subset.names = "nGene", low.thresholds = 500, high.thresholds = Inf)
Rep<-NormalizeData(Rep_filt,normalization.method = "LogNormalize",scale.factor = 10000)
Rep<-FindVariableGenes(Rep,mean.function = ExpMean,dispersion.function = LogVMR,do.plot=F)
                        # x.low.cutoff = 0.2,x.high.cutoff = 1.5,y.cutoff = -0.5)

# Select highly variable genes  
gene<-nrow(Rep@hvg.info)

i=500              
  if (gene >= i){
    top_gene=head(rownames(Rep@hvg.info),i)
 }else{
 top_gene=rownames(Rep@hvg.info)
      }

# standard scaling (no regression)
Rep_top<-Rep@raw.data[top_gene,]
Rep <-ScaleData(Rep,genes.use = top_gene,display.progress = F)

corrder<-strsplit(colnames(Rep_top),'x')
x<-sapply(corrder,'[',1)
y<-sapply(corrder,'[',2)
x<-as.numeric(x)  ### input data.frame is number matrix
y<-as.numeric(y)
info<-data.frame(x,y)

pp=pos2pp(info) 
## Convert cell positions(corrder) to point pattern,

pp=set_marks(pp,Rep_top,log.fcn = log10,pseudo.count = 1)

# topvar.genes =head(rownames((Rep@hvg.info),10))
# pp2plot = pp_select(pp, topvar.genes)
# plot_pp_scatter(pp, log_marks = FALSE, scale_marks = FALSE, pal.direction = -1)

t0<-Sys.time()
trendstat_list = trendsceek_test(pp,nrand=10000)
t1<-Sys.time()
t<-difftime(t1,t0,units = 'mins')

gene<-as.character(i)
time_min<-as.numeric(t)
test_time<-data.frame(gene,time_min)

alpha = 0.05   ##Benjamini-Hochberg
sig_list = extract_sig_genes(trendstat_list, alpha)

sig_gene_v<-sig_list$markvario$gene

sig_gene_c<-sig_list$markcorr$gene

sig_genes=union(sig_gene_v,sig_gene_c)

df_sig=data.frame(sig_gene=sig_genes)

write.csv(df_sig,paste0('results/BC',i,'_sig_gene_list.csv')
}